In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
from google.colab import drive

In [6]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Load the data
file_path = Path("/content/drive/My Drive/Colab Notebooks/resources/LoanStats_2019Q1.csv")
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

# Use Custom Encoding for date fields to convert to month since all are in the same year
months_convert = {
    "Jan-2019": 1,
    "Feb-2019": 2,
    "Mar-2019": 3,
    "Apr-2019": 4,
    "May-2019": 5
}
df['issue_d'] = df['issue_d'].apply(lambda x: months_convert[x])
df['next_pymnt_d'] = df['next_pymnt_d'].apply(lambda x: months_convert[x])

# Reset DataFrame Index
df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,3,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,3,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,3,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,3,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,3,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [8]:
# Create our target
y = df[['loan_status']].copy()

# Create our features
df = df.drop(columns='loan_status')

# Quantify non-numeric columns of data
df2 = pd.get_dummies(df)

# Scale the data to reduce the impact of large numeric data vs small
# In this case I chose to scale because interest rate is so impactful, along with debt to income
# but these numbers are far smaller than the loan amount requested by the borrower
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()
X_scaled = data_scaler.fit_transform(df2)
X = pd.DataFrame(X_scaled, columns=df2.columns)

In [9]:
# Display features DataFrame "X"
X.describe()

,loan_amnt,int_rate,installment,annual_inc,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,...,6.881700e+04,6.881700e+04,6.881700e+04,68817.0,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,68817.0,68817.0
mean,9.148043e-17,-1.057291e-16,-6.091812e-17,8.074232e-17,-5.286454e-17,-1.280313e-17,3.923540e-18,-1.610716e-17,-5.947260e-17,-4.119717e-17,...,4.749548e-17,1.398019e-16,4.047441e-17,0.0,1.734618e-17,-9.788199e-17,9.086092e-17,-3.799639e-17,0.0,0.0
std,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,...,1.000007e+00,1.000007e+00,1.000007e+00,0.0,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,0.0,0.0
min,-1.525462e+00,-1.407007e+00,-1.561349e+00,-7.628861e-01,-9.762264e-01,-1.078175e+00,-3.031425e-01,-6.564915e-01,-1.757869e+00,-3.742043e-01,...,-9.569400e-01,-7.729317e-01,-4.167860e-01,0.0,-3.760255e-01,-2.659394e+00,-2.481982e+00,-4.029039e-01,0.0,0.0
25%,-7.470458e-01,-8.231618e-01,-7.461036e-01,-3.306281e-01,-9.762264e-01,-3.905201e-01,-3.031425e-01,-6.564915e-01,-7.616592e-01,-3.742043e-01,...,-9.569400e-01,-7.729317e-01,-4.167860e-01,0.0,-3.760255e-01,3.760255e-01,4.029039e-01,-4.029039e-01,0.0,0.0
50%,-1.632334e-01,-2.019175e-01,-2.641560e-01,-1.316302e-01,3.681190e-01,-9.991302e-02,-3.031425e-01,-6.564915e-01,-2.635541e-01,-3.742043e-01,...,-9.569400e-01,-7.729317e-01,-4.167860e-01,0.0,-3.760255e-01,3.760255e-01,4.029039e-01,-4.029039e-01,0.0,0.0
75%,7.124852e-01,5.813905e-01,5.812919e-01,1.365843e-01,3.681190e-01,2.416864e-01,-3.031425e-01,6.625677e-01,5.666210e-01,-3.742043e-01,...,1.044998e+00,1.293775e+00,-4.167860e-01,0.0,-3.760255e-01,3.760255e-01,4.029039e-01,-4.029039e-01,0.0,0.0
max,2.269318e+00,3.754100e+00,4.150440e+00,7.535346e+01,1.712464e+00,4.837948e+01,2.475388e+01,5.938804e+00,9.864583e+00,1.150248e+01,...,1.044998e+00,1.293775e+00,2.399313e+00,0.0,2.659394e+00,3.760255e-01,4.029039e-01,2.481982e+00,0.0,0.0


In [10]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [11]:
# Import sklean function to split datasets into testing and training sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [12]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [13]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.797195732662936

In [14]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   63,    24],
       [ 2221, 14897]])

In [15]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.72      0.87      0.05      0.79      0.62        87
   low_risk       1.00      0.87      0.72      0.93      0.79      0.64     17118

avg / total       0.99      0.87      0.72      0.93      0.79      0.64     17205



In [16]:
# List the features sorted in descending order by feature importance
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.07336771398636914, 'total_rec_prncp'),
 (0.06265380706698574, 'last_pymnt_amnt'),
 (0.061214456735745174, 'total_rec_int'),
 (0.051186029668743825, 'total_pymnt_inv'),
 (0.04479227807771876, 'total_pymnt'),
 (0.03322467776357509, 'issue_d'),
 (0.03204431671775273, 'int_rate'),
 (0.019708244948559075, 'annual_inc'),
 (0.018610370805577255, 'installment'),
 (0.018428239738433404, 'out_prncp'),
 (0.0166597043921789, 'dti'),
 (0.016607774665930896, 'mo_sin_old_rev_tl_op'),
 (0.016382840128861088, 'mths_since_recent_inq'),
 (0.016006751228655015, 'total_bc_limit'),
 (0.015806643512237983, 'total_bal_il'),
 (0.015501260940533381, 'max_bal_bc'),
 (0.015388938494204633, 'bc_util'),
 (0.015194653058727792, 'out_prncp_inv'),
 (0.015099017543687915, 'bc_open_to_buy'),
 (0.01507237160484403, 'avg_cur_bal'),
 (0.014329451993364117, 'revol_bal'),
 (0.013268478636251546, 'total_rev_hi_lim'),
 (0.013075184395227866, 'il_util'),
 (0.012771795317291994, 'all_util'),
 (0.012648306932594142, 'mths_sin

In [22]:
# Train the EasyEnsembleClassifier
from sklearn.ensemble import AdaBoostClassifier
from imblearn.ensemble import EasyEnsembleClassifier, RUSBoostClassifier

base_estimator = AdaBoostClassifier(n_estimators=100)

#eec = EasyEnsembleClassifier(n_estimators=100, base_estimator=base_estimator, random_state=1)
#eec.fit(X_train, y_train)

rusboost = RUSBoostClassifier(n_estimators=100, base_estimator=base_estimator)
rusboost.fit(X_train, y_train)

RUSBoostClassifier(base_estimator=AdaBoostClassifier(n_estimators=100),
                   n_estimators=100)

In [23]:
# Calculated the balanced accuracy score
#y_pred = eec.predict(X_test)
y_pred = rusboost.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9526874312580829

In [24]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   82,     5],
       [  636, 16482]])

In [25]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.11      0.94      0.96      0.20      0.95      0.91        87
   low_risk       1.00      0.96      0.94      0.98      0.95      0.91     17118

avg / total       1.00      0.96      0.94      0.98      0.95      0.91     17205

